# Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import prince
import seaborn as sns
from mlxtend.plotting import plot_pca_correlation_graph
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
SEED = 0

In [ ]:
np.random.seed(SEED)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

var_categoriques_extra = ["NbAcceptedCampaigns", "HasAcceptedCampaigns", "NbChildren"]

var_categoriques_fe = var_categoriques + var_categoriques_extra

# Analyse multi-variée

## Analyse en Composantes Principales (ACP)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("scaler", StandardScaler(), var_numeriques),
    ],
)

In [ ]:
df_centre_reduit = pd.DataFrame(
    preprocessor.fit_transform(df), columns=df[var_numeriques].columns
)

In [ ]:
acp = PCA(random_state=SEED)

In [ ]:
acp.fit(df_centre_reduit)

In [ ]:
variance_expliquee = pd.Series(acp.explained_variance_ratio_)

In [ ]:
variance_expliquee

In [ ]:
variance_expliquee.plot.barh()

In [ ]:
composantes_principales = pd.DataFrame(
    acp.fit_transform(df_centre_reduit),
    index=df.index,
)

In [ ]:
composantes_principales.columns = [
    f"ACP{i+1}" for i in range(composantes_principales.shape[1])
]

In [ ]:
composantes_principales.head()

In [ ]:
sns.scatterplot(composantes_principales, x="ACP1", y="ACP2", hue=df["Response"])

### Cercle de corrélations

In [ ]:
plot_pca_correlation_graph(
    df_centre_reduit,
    df_centre_reduit.columns,
    X_pca=composantes_principales.iloc[:, :2],
    explained_variance=acp.explained_variance_[:2],
    dimensions=(1, 2),
)

In [ ]:
_, correlation_matrix = plot_pca_correlation_graph(
    df_centre_reduit,
    df_centre_reduit.columns,
    X_pca=composantes_principales.iloc[:, :4],
    explained_variance=acp.explained_variance_[:4],
    dimensions=(3, 4),
)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

## Analyse Factorielle des Correspondances (AFC)

### Kidhome vs Teenhome

In [ ]:
table_contingence = pd.crosstab(df["Kidhome"], df["Teenhome"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(random_state=SEED)

ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

### Statut marital et niveau d'éducation

In [ ]:
table_contingence = pd.crosstab(df["Marital_Status"], df["Education"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(random_state=SEED)
ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

In [ ]:
# todo: à interpréter

## Analyse des Correspondances Multiples (ACM)

### Variables qualitatives uniquement

In [ ]:
df["HasAcceptedCampaigns"] = df["HasAcceptedCampaigns"].astype(int)

In [ ]:
mca = prince.MCA(
    n_components=df[var_categoriques_fe].shape[1],
    random_state=SEED,
)
mca = mca.fit(df[var_categoriques_fe])

In [ ]:
composantes_acm = mca.row_coordinates(df[var_categoriques_fe])
composantes_acm.columns = [f"ACM{i+1}" for i in range(composantes_acm.shape[1])]

In [ ]:
mca.plot(df[var_categoriques_fe])

In [ ]:
mca.plot(df[var_categoriques_fe], x_component=2, y_component=3)

### Avec variables quantitatives en supplémenaire

In [ ]:
mca.plot(df[var_categoriques_fe + var_numeriques])

In [ ]:
# todo: à interpréter

### Essai sans Education

In [ ]:
mca = prince.MCA(random_state=SEED)
mca = mca.fit(df[var_categoriques_fe].drop(columns=["Education"]))

mca.plot(df[var_categoriques_fe].drop(columns=["Education"]))

# Sauvegarde des données

In [ ]:
composantes_principales.to_csv("data/composantes_acp.csv")
composantes_acm.to_csv("data/composantes_acm.csv")